In [1]:
!pip install torch transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 149.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 215.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 216.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 229.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 159.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 230.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 226.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 241.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 239.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 226.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
from huggingface_hub import login
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

#### Login to Hugging Face

In [6]:
#Login to hugging face

# Replace 'your_huggingface_token' with your actual token
token=os.environ["HF_TOKEN"]
login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [7]:
# We will use the default dataset of the project to download the model. But it can be any dataset you have access to
def get_download_dataset_folder(model_name):
    ds_dir = os.environ['DOMINO_DATASETS_DIR']
    ds_name = os.environ['DOMINO_PROJECT_NAME']
    download_ds_dir = f"{ds_dir}/{ds_name}/{model_name}"
    return download_ds_dir

In [8]:
# Save the model to a dataset
def download_model(model_name,save_path):
    print(f"Downloading {model_name} to location {save_path}")
    # Load the model and tokenizer from Hugging Face
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Save them locally
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"Model saved locally at {save_path}")

#### Make sure you accept the terms of service for the model you are downloading from HF

1. Download the `Mistral-7B-Instruct-v0.3` model
2. Download the `google/gemma-2b` model

In [ ]:

# Choose the model (base or instruct)
#model_name = "mistralai/Mistral-7B-Instruct-v0.3" 
#save_path = f"{download_ds_dir}/{model_name}"  # Set the directory to save the model

#download_model(model_name,save_path)

model_name = "google/gemma-2b"
save_path = get_download_dataset_folder(model_name)
download_model(model_name,save_path)


### Test the downloaded model locally

In [9]:


def load_local_llm(model_path):
    """Load a locally stored LLM model from disk."""
    #tokenizer = AutoTokenizer.from_pretrained(model_path)
    #model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")

    print(f"{model_name} loaded in float16 on CPU")
    return pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_text(llm_pipeline, prompt, max_length=50):
    """Generate text using the loaded LLM."""
    response = llm_pipeline(prompt, max_length=max_length, do_sample=True, temperature=0.7)
    return response[0]['generated_text']




In [10]:
model_name = "google/gemma-2b"
save_path = f"{download_ds_dir}/{model_name}"  # Set the directory to save the model

model_path = save_path  # Change this to your local model directory
model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             torch_dtype=torch.float16, 
                                             device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a text-generation pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time in a distant land,"
output = text_generator(prompt, max_length=50, do_sample=True)
print(output[0]['generated_text'])



Loading checkpoint shards: 100%|██████████| 3/3 [00:32<00:00, 10.91s/it]
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Once upon a time in a distant land, a young warrior was seeking a legendary amulet that would save his kingdom from the clutches of a bloodthirsty princess. After hours of searching, he found the amulet on the back of a giant spider and he
